# A Well Conditioned Estimator of Large Covariance Matrices

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from covShrinkage.linear import IdentityShrinkage, LinearShrinkage

In [3]:
n_vals = np.array([200, 160, 100, 80, 50, 40, 32, 25, 20, 16, 10, 8, 5, 4])
p_vals = np.array([4, 5, 8, 10, 16, 20, 25, 32, 40, 50, 80, 100, 160, 200])

In [4]:
def create_covariance(p: int) -> np.ndarray:
    """Create a covariance matrix with decaying eigenvalues."""
    lambdas = np.random.lognormal(mean=1, sigma=0.5, size=p)
    return np.diag(lambdas)

def create_data(n: int, p: int, sigma: np.ndarray) -> np.ndarray:
    """Create data from a multivariate normal distribution with given covariance."""
    return np.random.multivariate_normal(mean=np.zeros(p), cov=sigma, size=n)

In [5]:
np.random.seed(42)

n, p = 200, 4

trials = np.zeros((2000, 1))

for t in range(2000):
    sigma = create_covariance(p)
    X = create_data(n, p, sigma)

    sample_cov = X.T @ X / n

    # s_star = IdentityShrinkage(assume_centered=True).fit(X).covariance
    s_star = LinearShrinkage(assume_centered=True, rho=0.5, target=np.eye(p)).fit(X).covariance

    trials[t] = (np.linalg.norm(sigma - sample_cov, 'fro')**2 - np.linalg.norm(sigma - s_star, 'fro')**2) / np.linalg.norm(sigma - sample_cov, 'fro')**2

In [6]:
np.mean(trials), np.std(trials)

(np.float64(-7.126337507673902), np.float64(6.6747971067094385))

In [7]:
import numpy as np


def cov1para(Y, k=None):
    """
    Ledoit-Wolf linear shrinkage estimator for covariance matrix.
    
    Shrinks towards one-parameter matrix:
        - all variances of the target are the same
        - all covariances of the target are zero
    
    Parameters
    ----------
    Y : ndarray of shape (N, p)
        Raw data matrix of N iid observations on p random variables
    k : int, optional
        - If None (default): demeans the data and adjusts sample size
        - If 0: no demeaning takes place
        - If 1: data is already demeaned
    
    Returns
    -------
    sigmahat : ndarray of shape (p, p)
        Invertible covariance matrix estimator
    
    References
    ----------
    Ledoit, O. and Wolf, M. (2004). A well-conditioned estimator for 
    large-dimensional covariance matrices. Journal of Multivariate Analysis.
    
    This implementation is based on the MATLAB code (version 01/2021)
    by Olivier Ledoit and Michael Wolf.
    """
    # Get sample size and matrix dimension
    N, p = Y.shape
    
    # De-mean returns if required
    if k is None or np.isnan(k):
        # Default setting: demean the data
        Y = Y - np.mean(Y, axis=0)
        k = 1  # subtract one degree of freedom
    
    # Adjust effective sample size
    n = N - k
    
    # Compute sample covariance matrix
    sample = (Y.T @ Y) / n
    
    # Compute shrinkage target (scaled identity matrix)
    meanvar = np.mean(np.diag(sample))
    target = meanvar * np.eye(p)
    
    # Estimate the parameter pi (Ledoit and Wolf, 2003)
    Y2 = Y ** 2
    sample2 = (Y2.T @ Y2) / n  # sample matrix of squared returns
    piMat = sample2 - sample ** 2
    pihat = np.sum(piMat)
    
    # Estimate the parameter gamma (Ledoit and Wolf, 2003)
    gammahat = np.linalg.norm(sample - target, 'fro') ** 2
    
    # Parameters rho_diag and rho_off (set to 0 in this implementation)
    rho_diag = 0
    rho_off = 0
    
    # Compute shrinkage intensity
    rhohat = rho_diag + rho_off
    kappahat = (pihat - rhohat) / gammahat
    shrinkage = max(0, min(1, kappahat / n))
    
    # Compute shrinkage estimator
    sigmahat = shrinkage * target + (1 - shrinkage) * sample
    
    return sigmahat


In [8]:
np.random.seed(0)

n, p = 200, 4

trials = np.zeros((2000, 1))

for t in range(2000):
    sigma = create_covariance(p)
    X = create_data(n, p, sigma)

    sample_cov = X.T @ X / n

    # s_star = IdentityShrinkage(assume_centered=True).fit(X).covariance
    s_star = cov1para(X, k=1)

    trials[t] = (np.linalg.norm(sigma - sample_cov, 'fro')**2 - np.linalg.norm(sigma - s_star, 'fro')**2) / np.linalg.norm(sigma - sample_cov, 'fro')**2

In [10]:
np.mean(trials)

np.float64(0.034624386606333914)